# 不同產品線的概念設計智能客服

- 目前痛點
    - 目前為止，每次都要重新 embedding
    - 重新做 embedding 的時間很久，如何不重新 embedding
    - 如何把不同產品線的 embedding 資料分開
- 測試 AI Agent 的三步驟
    1. 收斂關鍵場景
        - 與業務部門列出 100–200 條「最怕答錯」 的情境（合規條文、價格算法、治療指南…）。
    2. 生成＋人工校對
        - 利用文件透過 AI 來 做一個 Q&A 生成器，再由客戶的專家花大量時間確認答案。
    3. 持續增補
        - 將線上失敗案例自動寫回題庫，達到「用真實錯誤反餵模型」的循環。

## 不重新 embedding 的方式
- `qdrant_init.py`
### 安裝環境
1. `poetry new week5`
2. `cd week5` ，記要到 toml 檔裡切成 3.11 以上版本的 python。
3. `poetry add langchain langchain-openai langchain-community qdrant-client`
- 如果是打開程式碼壓縮檔的同學，可以使用 `poetry install` 來把環境安裝起來。

In [ ]:
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant # <--
from langchain_openai import AzureOpenAIEmbeddings

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

client = QdrantClient(url="http://localhost:6333")

collection_name = "lyrics"
# use collecion name to assign the specific colletion in Qdrant
qdrant = Qdrant(client, collection_name, embeddings_model)

result = qdrant.similarity_search_with_score(query = "工程師寫城市", k =2)

print(result)

- 如果之前的 lyrics 資料已經刪了的話，用加上下面這段，把資料弄回來

In [ ]:
data_objs = [
    {
        "id": 1,
        "lyric": "我會披星戴月的想你，我會奮不顧身的前進，遠方煙火越來越唏噓，凝視前方身後的距離"
    },
    {
        "id": 2,
        "lyric": "而我，在這座城市遺失了你，順便遺失了自己，以為荒唐到底會有捷徑。而我，在這座城市失去了你，輸給慾望高漲的自己，不是你，過分的感情"
    }
]

lyric_list = [data_obj["lyric"] for data_obj in data_objs]

qdrant = Qdrant.from_texts(
    lyric_list,
    embeddings_model,
    url="localhost", 
    collection_name="lyrics",
    force_recreate=True,
)

## Filter
- 沒有 subsidy_qa 的 collection 的話，去把上周的 07_store.py 跑起來。
- 檔名：`03_qdrant_filter.py`
- 透過 `metadata` 來篩資料
- Filter Condition: https://qdrant.tech/documentation/concepts/filtering/#filtering-conditions

In [ ]:
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore # <--
# from langchain_qdrant import Qdrant # <- (deprecated)
from qdrant_client.http import models
from langchain_openai import AzureOpenAIEmbeddings

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

client = QdrantClient()
collection_name = "subsidy_qa"
qdrant = QdrantVectorStore(client, collection_name, embeddings_model) # <--
# qdrant = Qdrant(client, collection_name, embeddings_model) # <-

result = qdrant.similarity_search_with_score(
    query = "申請後多久可以領到補助", 
    k = 3 , 
    filter=models.Filter( # 注意此 !!! 可以篩 metadata 中的資料 (注意此 model 是 Qdran 原生寫法)
        must=[
            models.FieldCondition(
                key="metadata.page", match=models.MatchValue(value=6,) # 只找 page 6 的資料
                # key="metadata.author", match=models.MatchValue(value="蔡宜晴",)
            ),
        ]
    )
)

print(result)

## Custom Payload
- 檔名：`04_custom_payload.py`
- `poetry add pypdf`

In [ ]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("../MRT.pdf")

docs = loader.load()

splitter = RecursiveCharacterTextSplitter(    
    chunk_size=500,
    chunk_overlap=100)

chunks = splitter.split_documents(docs)

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

metadatas = []
for text in chunks:
    # 客製 metadata (會覆蓋)
    metadatas.append({
        "department": "MRT",
        # "source": text.metadata["source"],
        # "page": text.metadata["page"]
    })


qdrant = QdrantVectorStore.from_texts(
    [t.page_content for t in chunks],
    embeddings_model,
    url="localhost", 
    collection_name="mrt",
    force_recreate=True,
    metadatas = metadatas
)

## 整合在一起
- 檔名：`05_combine.py`
- RAG 資料量少的話，可以用考慮用 collection 做切換（公司永遠不會有新客戶）
- RAG 資料量多的話，建議使用 payload (metadata) 的特性
  - Vector DB 的效能差異，建議不要多件 collection，透過 payload 來區分資料

In [ ]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client.http import models

loader = PyPDFLoader("../MRT.pdf")

docs = loader.load()

splitter = RecursiveCharacterTextSplitter(    
    chunk_size=500,
    chunk_overlap=100)

chunks = splitter.split_documents(docs)

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

# MRT Department
metadatas = []
for text in chunks:
    metadatas.append({
        "department": "MRT",
        "source": text.metadata["source"],
        "page": text.metadata["page"]
    })

qdrant = QdrantVectorStore.from_texts(
    [t.page_content for t in chunks],
    embeddings_model,
    url="localhost", 
    collection_name="my_company",
    force_recreate=True,
    metadatas = metadatas
)

result = qdrant.similarity_search_with_score(
    query = "兒童票", 
    k = 1 , 
    filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.department", match=models.MatchValue(value="MRT",)
            ),
        ]
    )
)
print(result)

# Children Department
loader = PyPDFLoader("../Children.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(    
    chunk_size=500,
    chunk_overlap=100)

chunks = splitter.split_documents(docs)

metadatas = []
for text in chunks:
    metadatas.append({
        "department": "Children",
        "source": text.metadata["source"],
        "page": text.metadata["page"]
    })


qdrant = QdrantVectorStore.from_texts(
    [t.page_content for t in chunks],
    embeddings_model,
    url="localhost", 
    collection_name="my_company",
    metadatas = metadatas
        # 特別注意這裡不要再 recreate True
)

result = qdrant.similarity_search_with_score(
    query = "兒童", 
    k = 1 , 
    filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.department", match=models.MatchValue(value="Children",)
            ),
        ]
    )
)

print(result)

[(Document(metadata={'department': 'MRT', 'source': '../MRT.pdf', 'page': 3, '_id': 'cffc19f9-d5d3-4ace-a9fe-c1918c432d72', '_collection_name': 'my_company'}, page_content='進入付費區至離開付費區之停留時限規定如下： \n（一） 不同一車站進出，最大時限為 2 小時； \n（二） 同一車站進出，最大時限為 15 分鐘； \n違反前項規定者，除應付之票價外，應於車站詢問處繳交或\n由驗票閘門自動扣除本公司公告之單程票最低票價之金額。 \n十四、旅客乘車應支付之票價，以本公司於車站公告之票價表為準。\n其運價一律全票收費。同站進出者，應支付車站公告最低單程\n票票價。如屬特殊狀況（民眾借用廁所等），可洽站務人員協\n助處理。 \n未滿 6 歲之兒童（身高滿 115 公分應出示身分證明）、身高未\n滿 115 公分之兒童，得由購票旅客陪同免費乘車。每 1 位購票\n旅客最多以陪同 4 名為原則，並妥善照護其安全。 \n十五、 旅客無票、持用失效車票或冒用不符身分之車票乘車者，除 \n補繳票價外，並支付票價50倍之違約金。 \n前項應補繳票價及支付之違約金，如旅客不能證明其起站地點\n者，以本公司公告之單程票最高票價計算。 \n十六、 旅客持用偽造或變造之車票，除依前點無票規定處理外，並 \n依法移送偵辦。'), 0.46188802)]
[(Document(metadata={'department': 'Children', 'source': '../Children.pdf', 'page': 4, '_id': 'aa7149ea-da09-4e15-97e5-fe2677720137', '_collection_name': 'my_company'}, page_content='檢具相關佐證資料，向核定機關提起申復。逾期提出申復者，不\n予受理。 \nQ5 兒童原本自己照顧並領取未滿 2 歲育兒津貼，後來改送托與政府簽約托嬰\n中心或居家托育人員(保母），或者原領托育補助改領育兒津貼，需要 \n重新提出申請嗎？ \nA5 一、兒童原本領取未滿2歲育兒津貼，如改送托與政府簽約的托嬰中心或

### 整合 RAG
- 檔名：`06_rag.py`
- Qrant doc: https://qdrant.tech/documentation/concepts/
- Azure AI Search 可以協助處理進階 RAG 的問題，但貴
- 圖片也可以放到向量資料庫，並透過 metadata 作為搜尋的條件
- 利用多模態模型將圖片 (表格) 轉成文字，再 embedding 後放到向量資料庫
- RAG 搭配外部網頁搜尋 eg. `gpt search`

In [6]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from qdrant_client import QdrantClient
from langchain_qdrant import Qdrant
from qdrant_client.http import models
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

client = QdrantClient()
collection_name = "my_company"
# 不成新 embedding
qdrant = Qdrant(client, collection_name, embeddings_model)

# as_retriever 轉 Vector DB 型
retriever = qdrant.as_retriever(search_kwargs=dict( # <-- 一定要先轉成 dict
                filter = models.Filter( # 才能用 Qdrant 原生的 Model
                    must=[
                        models.FieldCondition(
                            key="metadata.department",
                            match=models.MatchValue(value="Children")
                            # match=models.MatchValue(value="MRT")
                        )
                    ]
                )
            ), k=3)

model = AzureChatOpenAI(
	api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",
    openai_api_version="2024-10-21",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://tibamesktest.openai.azure.com/",
    temperature=0,
)

prompt = ChatPromptTemplate.from_template("""請回答依照 context 裡的資訊來回答問題:
<context>
{context}
</context>
Question: {input}""")


document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# response = retrieval_chain.invoke({"input": "請問幾歲兒童不用票"})
response = retrieval_chain.invoke({"input": "弱勢家庭的補助有多少"})

print(response["answer"])


根據你提供的 context，**弱勢家庭**未滿2歲兒童的托育補助金額如下（單位：元/月）：

| 出生排序   | 公共化托育（公設民營、社區公共托育家園） | 準公共托育（簽約私立托嬰中心及居家托育） |
|------------|------------------------------------------|------------------------------------------|
| 第1名子女  | 11,000                                   | 17,000                                   |
| 第2名子女  | 12,000                                   | 18,000                                   |
| 第3名以上  | 13,000                                   | 19,000                                   |

**說明：**
- 弱勢家庭包括：家有未滿2歲之發展遲緩或身心障礙幼兒、特殊境遇家庭、經政府認定之弱勢家庭。
- 若實際支付托育費用低於上述標準，則以實際支付金額為準。

**舉例：**
- 如果是弱勢家庭的第一名子女，送公共化托育，每月補助11,000元；送準公共托育，每月補助17,000元。
- 第二名子女則分別為12,000元（公共化）和18,000元（準公共）。
- 第三名以上子女則分別為13,000元（公共化）和19,000元（準公共）。


# FastAPI 入門
- 什麼是 FastAPI？
  - 輕量級的web框架（相較於 Django）
  - 性能良好，據稱某些狀況下可以比擬 Golang
  - 使用 OpenAPI（以前稱為 Swagger）和 JSON Schema 標準，FastAPI 自動生成文檔，可以通過瀏覽器訪問
  - LangChain 與 FastAPI 大力整合中
  - `Starlette`：是一個輕量級的 ASGI（Asynchronous Server Gateway Interface）框架。（FastAPI 一直不上 1.0 版的主因，因為 Starlette 還未上 1.0）
  - `Pydantic`：是一個數據驗證和設置管理工具，使用 Python 的類型提示來驗證數據。它讓 FastAPI 可以自動驗證客戶端發來的數據，並將其序列化/反序列化成 Python 物件，這大大簡化了數據處理和 API 開發過程
- 檔名：`07_fastapi.py`
- `poetry add fastapi uvicorn`
- 可透過 http://192.168.70.128:8000/redoc or http://192.168.70.128:8000/doc 開啟 swagger 測試頁面
  - `app = FastAPI(docs_url=None, redoc_url=None)`

In [ ]:
# run on cmd: `uvicorn <file_name_without_.py>:app --reload --host 0.0.0.0 --port 5050`
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional
## 跨來源資源共享（CORS）是一種基於HTTP 標頭的機制，
## 允許伺服器指示瀏覽器允許從除其自身以外的任何來源（域名、協定或通訊埠）加載資源
# from fastapi.middleware.cors import CORSMiddleware # <--

app = FastAPI()
# app = FastAPI(docs_url=None, redoc_url=None) # 關閉 API 測試頁面

# 作部分權限管控 (白名單)
# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["https://yourapp.com", "https://www.yourapp.com"],
#     allow_credentials=True,
#     allow_methods=["GET", "POST", "PUT", "DELETE"],
#     allow_headers=["X-Custom-Header", "Authorization"],
# )

@app.get("/")
async def read_root():
    return {"Hello": "World"}

# get method
# http://localhost:8000/items/5?name=example
@app.get("/items/{item_id}")
async def read_item(item_id: int, name: str):
	return {"item_id": item_id, "name": f"{name}, Love you"}

class Item(BaseModel):
    name: str
    description: Optional[str] = None
    price: float
    tax: Optional[float] = None

# post method
# http://localhost:8000/items/
@app.post("/items/")
async def create_item(item: Item):
    return {"name": item.name, "price": item.price, "description": item.description, "tax": item.tax}

- `uvicorn week5.07_fastapi:app --reload`
  - ( ctrl +c 可以離開)
- 開新的 terminal 分頁，用下面指令做請求
```bash
curl -X 'POST' 'http://localhost:8000/items/' -H 'accept: application/json' -H 'Content-Type: application/json' -d '{
  "name": "很貴的火雞肉飯",
  "description": "火雞肉飯加醬油",
  "price": 105.0,
  "tax": 5.0
}'
```

# LangServe
- 把 LangChain 包裝給 FastAPI (WebAPI)
- 把 LangChain 的 Runnable 和 Chains 部署成 REST API 的工具。
- 用很簡單的方式把 LangChain 和 FastAPI 整合起來。
- 之前的 Pydantic 會造成的文件問題，已經在 LangChain 0.3 改善
## LangChain 整合 FastAPI
- `poetry --version`: version 1.6.1
- `poetry add langchain-cli` 
- 在安裝 langchain-cli 時，有可能會跳出 uvicorn 的版本的問題，這時候我們去 pyproject.toml 裡，把 uvicorn 那裡，設定為 uvicorn = "0.23.2"，fastapi = "^0.110.0"，再 `poetry update`
- `langchain app new week5app`，terminal 提示按 enter 跳過。
- 然後先用指令 `exit` 離開現在的虛擬環境
- `cd week5app`，進去後再一次 `poetry install`，然後再把該安裝的套件安裝上 `poetry add langchain-openai langchain-community qdrant-client langchain-qdrant`
- 接著可能出現  pydantic 要 >=2.7.4 的錯誤提示，我們進到 week5app 的 toml 檔，把 pydantic = "<2" 改成 pydantic = ">2"，然後再安裝一次。
- 再 `poetry shell` 進到我們這個 langserve 的虛擬環境中。
- 新增 `rag` folder 並加檔案檔名：`rag/rag_chain.py`

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http import models
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from pydantic import BaseModel
    
embeddings_model = AzureOpenAIEmbeddings(
    api_key="xx",
    azure_deployment="text-embedding-3-small", 
    openai_api_version="2024-06-01",
    azure_endpoint="https://kokotest3.openai.azure.com/",
)

client = QdrantClient()
collection_name = "my_company"
qdrant = QdrantVectorStore(client, collection_name, embeddings_model)

retriever = qdrant.as_retriever(search_kwargs=dict(
				filter=models.Filter(
					must=[
						models.FieldCondition(
							key="metadata.department",
							match=models.MatchValue(value="Children")
						)
					]
				)
			))

model = AzureChatOpenAI(
	api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",
    openai_api_version="2024-10-21",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://tibamesktest.openai.azure.com/",
    temperature=0,
)

prompt = ChatPromptTemplate.from_template("""請回答依照 context 裡的資訊來回答問題:
<context>
{context}
</context>
Question: {input}""")


document_chain = create_stuff_documents_chain(model, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Add typing for input (整合 LangSrv 要做的事)
class Question(BaseModel):
    input: str

rag_chain = retrieval_chain.with_types(input_type=Question)

- 新增並修改內容：`rag/__init__.py`
  - 有此程式，python 預設會識別為一個套件，可留白
```python
from rag.rag_chain import rag_chain
__all__ = ["rag_chain"]
```

- 新增並修改 app 資料夾中的`server.py`

In [ ]:
from fastapi import FastAPI
from fastapi.responses import RedirectResponse
from langserve import add_routes
from rag import rag_chain

app = FastAPI()


@app.get("/")
async def redirect_root_to_docs():
    return RedirectResponse("/docs")


# Edit this to add the chain you want to add
add_routes(app, rag_chain, path="/rag")

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)


- 啟動 LangServe `langchain serve`
- invoke 這個路由就是把 API 叫起來的方式，也可以改成 stream 來試試
```bash
curl --location --request POST 'http://localhost:8000/rag/invoke' \
--header 'Content-Type: application/json' \
--data-raw '{
    "input": {
        "input": "父母離婚，該由誰申請補助"
    }
}'
```
```bash
curl --location --request POST 'http://localhost:8000/rag/stream' \
--header 'Content-Type: application/json' \
--data-raw '{
    "input": {
        "input": "父母離婚，該由誰申請補助"
    }
}'
```

- 可以進到 play ground 來試玩
- http://localhost:8000/rag/playground/
- PS. 新的 chat playground 超難用，不如串到自己的系統做 UI。

### 客製化自己的參數
- Auth 相關: https://github.com/langchain-ai/langserve/tree/main/examples/auth/per_req_config_modifier
- Config 相關: https://github.com/langchain-ai/langserve/tree/main/examples/configurable_chain

# 補充資料，Agentic RAG 整合進 LangServe，再加上庫存的 tool call
- 把 RAG 給 Agent 化
1. 新增檔案 `rag/rag_agent.py`

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain_qdrant import QdrantVectorStore
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda
from langchain.tools import StructuredTool
import requests

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-small", 
    azure_endpoint = "https://tibamesktest.openai.azure.com", 
    api_key="2Dvpc8O3XBzbvAXNWpHuYlsNGHNf4Yec8RXDuDpXyDufwaYPMUwxJQQJ99BFACYeBjFXJ3w3AAABACOGl2BL",  
    api_version="2024-10-21"
)

client = QdrantClient()
collection_name = "my_company"
vectorstore = QdrantVectorStore(client, collection_name, embeddings_model)

retriever = vectorstore.as_retriever(
    search_kwargs=dict(
        filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="metadata.department",
                    match=models.MatchValue(value="Children"),
                )
            ]
        )
    )
)

# 把 RAG 封裝成 Tool 
rag_tool = create_retriever_tool(
    retriever=retriever,
    name="company_policy_search",
    description="查詢孩童托育補助 FAQ，回答與父母離婚或領養等問題相關的政策與金",
)

# 加上庫存 tool
class ProdInput(BaseModel):
    prod: str = Field(description="要查詢庫存的產品名稱")

def inventory_request(prod: str):
    resp = requests.get("https://koko-demo-api.azurewebsites.net/inventory", params={"prod": prod})
    resp.raise_for_status()
    return resp.json()

inventory_tool = StructuredTool.from_function(
    name="inventory_query",
    description="查詢產品的庫存",
    func=inventory_request,
    args_schema=ProdInput,
)

system_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是公司福利與補助專家。當需要查詢文件時，請呼叫 company_policy_search，當要查詢庫存時請使用inventory_query；若答案不在文件中，告知使用者『根據現有資料無相關資訊』。{agent_scratchpad}",
        ),
        ("user", "{input}"),
    ]
)

model = AzureChatOpenAI(
    api_key="xx",
    openai_api_version="2024-06-01",
    azure_deployment="gpt-4.1",
    azure_endpoint="https://kokotest3.openai.azure.com/",
    temperature=0,
)

agent = create_openai_functions_agent(llm=model, tools=[rag_tool, inventory_tool], prompt=system_prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[rag_tool, inventory_tool],
    verbose=True,
)


class Question(BaseModel):
    """LangServe 會自動在 /docs 顯示此欄位"""
    input: str = Field(..., description="使用者提問的句子")
class Answer(BaseModel):
    output: str
 
clean_executor = agent_executor | RunnableLambda(lambda r: {"output": r["output"]})
rag_agent = clean_executor.with_types(input_type=Question, output_type=Answer)


2. 修改 rag 資料夾的 `__init__.py` 

In [ ]:
from rag.rag_agent import rag_agent   
from rag.rag_chain import rag_chain

__all__ = ["rag_chain", "rag_agent"]

3. 在 `app/server.py` 裡加上 `add_routes(app, rag_agent, path="/rag_agent")`
4. `langchain serve `，然後
```bash
curl -X POST http://localhost:8000/rag_agent/invoke \
    -H "Content-Type: application/json" \
    -d '{"input": {"input": "父母離婚，該由誰申請補助？"}}'
    

curl -X POST http://localhost:8000/rag_agent/invoke \
    -H "Content-Type: application/json" \
    -d '{"input": {"input": "火雞肉飯還剩幾碗"}}'
```

# 作業
- 內容與第二周作業接近，但是需改用 LangChain 的方式來實作。
    1. 請依課堂上教過的內容，使用 Docker 自行在本地端架設 Qdrant 向量資料庫。
    2. 寫一隻 Python 程式「chain.py」，先把產品介紹的 TXT 檔讀取出來，再轉換成向量，存入 Qdrant 之中。接著建構好產品客服的 Chain，能輸入 prompt 並把產品資訊取出來。
    3. 把 chain.py 用 LangServe 部署成 web api。

- 評分標準：
    1. 程式碼邏輯正確。（兩個部份分別各 30 分）
    2. 程式碼可以順利跑起來。（兩個部分別各 20 分）

- 上傳作業說明：
    1. 截兩部份的圖片，第一部份是 chain. Py 的程式碼，第二部份是架成 web api 服務的程式碼。
    2. 截圖 web API 程式碼跑起來的結果畫面。
    3. 截圖必需要能看清楚與完整程式碼，並且檔名要說明該圖片是什麼內容。
    4. 壓縮後的資料夾檔名請統一命名：學號_作業名稱，zip 需學員座號作為辨識是否有繳交的評分結果